In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV,Lasso, LassoLarsCV, LinearRegression,RANSACRegressor
from sklearn.model_selection import cross_val_score,KFold,train_test_split,cross_val_predict, StratifiedKFold
from sklearn.metrics import *
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.ensemble import *
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from pandas.tools.plotting import scatter_matrix
import statsmodels.formula.api as sm
from scipy.stats import norm
from statsmodels.stats.weightstats import _zconfint_generic, _tconfint_generic
import datetime 
from  datetime  import datetime as dt 
from scipy import stats
import operator
from sklearn import cross_validation, grid_search, linear_model, metrics

from sklearn.feature_selection import VarianceThreshold
import re
import warnings 
warnings.filterwarnings("ignore")
import seaborn as sns
import matplotlib

import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr


%config InlineBackend.figure_format = 'png' #set 'png' here when working on notebook
%matplotlib inline

C:\Program Files\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Program Files\Anaconda2\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
data = pd.read_csv('dataset1707.csv', sep=';') #
data = data.loc[data.IS_FIRST==0]
agg68 = pd.read_csv('done686_old.csv', sep=';') 
agg68.drop([x for x in agg68.columns if x.endswith('_median')], inplace=1, axis =1)

data = data.loc[(data['TMP_STP_ALL_AVG']>360)&(data['TMP_STP_ALL_AVG']<700)&(data.TEMP_START_PHASE1!=0)&((data.ADAPTATION_KEY==190409109015)|(data.ADAPTATION_KEY==190309109015))]

data.drop(['TMP_STRT_HED_AVG_M','TMP_STRT_TAIL_AVG_M', 'PROFILE_SDV', u'PROFILE_MIN', u'PROFILE_AVG', u'PROFILE_MAX'], axis=1, inplace=True)
data = data.dropna()

In [3]:
data['Vol'] = data['PLT_TCK']*data['PLT_WID']*data['PLT_LEN']*1e-09
data['MEAS_TIME'] = data['MEAS_TIME'].apply(lambda x: dt.strptime(x, "%d.%m.%Y %H:%M:%S"))
data['PREV_MEAS_TIME'] = data['PREV_MEAS_TIME'].apply(lambda x: dt.strptime(x, "%d.%m.%Y %H:%M:%S"))

data['hour'] = 0
data['hour'] = data['MEAS_TIME'].apply(lambda x: x.hour)


data['diff_mins'] = data['MEAS_TIME'] - data['PREV_MEAS_TIME']
data['diff_mins'] = data['diff_mins'].apply(lambda x: round(x.total_seconds()/60,2))
# data['weekday'] = data['MEAS_TIME'].apply(lambda x: x.weekday())

data['prev_fail'] = data['NOT_IN_25_0C'].shift().fillna(0) 
data['CR_PREV'] = (data['CR_ACT'] - data['CR']).shift().fillna((data['CR_ACT'] - data['CR']).median())  
data['TMP_STP_ALL_PREV'] = data['TMP_STP_ALL_AVG'].shift().fillna(data.TMP_STP_ALL_AVG.median())  

data['ADAPT_TS_MEAN_PREV'] = data['ADAPT_TS_MEAN'].shift().fillna(data.ADAPT_TS_MEAN.median())  

data['ADAPT_CR_PREV'] = data['ADAPT_CR'].shift().fillna(data.ADAPT_CR.median())  

data['daypart'] = 0
data.ix[(data.hour>8)&(data.hour<=12), 'daypart'] = 1
data.ix[(data.hour>12)&(data.hour<=16), 'daypart'] = 2
data.ix[(data.hour>16)&(data.hour<=20), 'daypart'] = 3
data.ix[(data.hour>20)&(data.hour<=23), 'daypart'] = 4



for col in [u'WATER_FLOW_SPRAY_TOP', u'WATER_FLOW_SPRAY_BOTTOM']:
    data[col] = data[col].apply(lambda x: x.split('|'))
    data.ix[data[col].apply(lambda x: len(x))>4, col] = data[col].apply(lambda s: map('.'.join, zip(s[::2], s[1::2])))
    
for col in [ u'WATER_FLOW_LAM_INNER_TOP', u'WATER_FLOW_LAM_OUTER_TOP',
       u'WATER_FLOW_LAM_INNER_BOT', u'WATER_FLOW_LAM_OUTER_BOT']:
    data[col] = data[col].apply(lambda x: x.split('|'))
    data.ix[data[col].apply(lambda x: len(x))>36, col] = data[col].apply(lambda s: map('.'.join, zip(s[::2], s[1::2])))
    
for col in [ 'EDGE_MASKING_POS_DRIVE_SIDE', 'EDGE_MASKING_POS_OP_SIDE', 'BANKS_HEIGHT']:
    data[col] = data[col].apply(lambda x: x.split('|'))
    data.ix[data[col].apply(lambda x: len(x))>9, col] = data[col].apply(lambda s: map('.'.join, zip(s[::2], s[1::2])))
    
    
for col in [u'WATER_FLOW_SPRAY_TOP', u'WATER_FLOW_SPRAY_BOTTOM', u'WATER_FLOW_LAM_INNER_TOP', u'WATER_FLOW_LAM_OUTER_TOP',
       u'WATER_FLOW_LAM_INNER_BOT', u'WATER_FLOW_LAM_OUTER_BOT','EDGE_MASKING_POS_DRIVE_SIDE', 'EDGE_MASKING_POS_OP_SIDE', 'BANKS_HEIGHT'] :
    data[col] = data[col].apply(lambda x: [float(i) for i in x])
    

    
for col in [u'WATER_FLOW_SPRAY_TOP', u'WATER_FLOW_SPRAY_BOTTOM', u'WATER_FLOW_LAM_INNER_TOP', u'WATER_FLOW_LAM_OUTER_TOP',
       u'WATER_FLOW_LAM_INNER_BOT', u'WATER_FLOW_LAM_OUTER_BOT','EDGE_MASKING_POS_DRIVE_SIDE', 'EDGE_MASKING_POS_OP_SIDE', 'BANKS_HEIGHT'] :
    data[col] = data[col].apply(lambda x: np.mean(x))
    
    
    
for col in ['SIDE_SPRAY', 'AIR_BLOWER'] :
    data[col] = data[col].apply(lambda x: str(x.count('0')))
    


agg68.rename(columns={'pltid':'PIECE_ID'}, inplace=1)

df = pd.merge(agg68, data, on='PIECE_ID')    



# df['outcome'] = 0
# df.ix[(df.NOT_IN_40_0C==1)&(df.TMP_STP_ALL_AVG>510), 'outcome']=1

# df.ix[(df.NOT_IN_40_0C==1)&(df.TMP_STP_ALL_AVG<450), 'outcome']=2

# df.ix[(df.NOT_IN_25_0C==1)&(df.TMP_STP_ALL_AVG>480), 'outcome']=1

# df.ix[(df.NOT_IN_25_0C==1)&(df.TMP_STP_ALL_AVG<480),'outcome']=2

flowsum_cols = [x for x in df.columns if ('flow' in x or 'SpecFlw' in x ) and ('sum' in x or 'SUM' in x)]
df[flowsum_cols] = df[flowsum_cols].apply(lambda x: (x/60)*df['time'])
df.fillna(0, axis=1, inplace=1)

flow_cols = [x for x in data.columns if 'Flw' in x and ('WatAbs' not in x) and ('Ratio' not in x)]

df['Specflow_bot_sum_diff'] =  df['SCC._SccRtData.SccRtData.SpecFlw_Bot_SUM_mean'] - df[ 'SCC._SccRtData.SccRtData.SpecFlw_Bot_SUM_task_mean']
df['Specflow_top_sum_diff'] = df['SCC._SccRtData.SccRtData.SpecFlw_TOP_SUM_mean'] - df['SCC._SccRtData.SccRtData.SpecFlw_TOP_SUM_task_mean']

df['BHC_cen_flow_diff'] = df['BHC_cen_flow_sum_act_mean'] - df['BHC_flow_cen_sum_task_mean']
df['BHC_out_flow_diff'] = df[ 'BHC_out_act_sum_mean'] - df[ 'BHC_flow__out_sum_task_mean']
df[ 'THC_cen_flow_diff'] = df['THC_cen_flow_sum_act_mean'] - df['THC_flow_cen_sum_task_mean']
df['THC_out_flow_mean'] = df['THC_out_flow_sum_act_mean'] - df['THC_out_flow_sum_task_mean']

In [4]:
todrop = ['GFC.Ai._aiGfcPrsAirBlw_mean', 'GFC._Gfc2Hmi.Gfc2Hmi.GfcHmiTmpWatScc_ActV_mean','GFC._Gfc2Hmi.Gfc2Hmi.GfcHmiBtmTopFlwRatioSetMod_ActV_mean',
        'GFC._Hmi2Gfc.Hmi2Gfc.GfcHeadMskFlwTop_SetV_mean',
 'GFC._Hmi2Gfc.Hmi2Gfc.GfcHeadMskLenTop_SetV_mean',
 'GFC._Hmi2Gfc.Hmi2Gfc.GfcHeadMskFlwBot_SetV_mean',
 'GFC._Hmi2Gfc.Hmi2Gfc.GfcHeadMskLenBot_SetV_mean',
 'GFC._Hmi2Gfc.Hmi2Gfc.GfcTailMskFlwTop_SetV_mean',
 'GFC._Hmi2Gfc.Hmi2Gfc.GfcTailMskFlwBot_SetV_mean', 'THC._ThcRtData.ThcRtData.WatAbsFlwSum_mean', 
         'PRC.GlobVar.PrcLocRtData.PrcPosMean_mean',

 'EMC.GlobVar.EmcLocRtData.Mot_PosMean_mean',
 'EMC.GlobVar.EmcLocRtData.Mot_Pos_DsOsdiff_sum_mean',
 'EMC.GlobVar.EmcLocRtData.Mot_PosDiff_sum_mean', 'dist_from_plt_sum_mean', 'PPS_PIECE_ID',
 'PREV_PIECE_ID',
 'PIECE_ID_1',
 'PIECE_ID_2','PLT_TCK',
 'PLT_WID',
 'PLT_LEN','Vol', 'TMP_STP_HEAD_AVG',
 'TMP_STP_HEAD_MIN',
 'TMP_STP_HEAD_MAX',
 'TMP_STP_HEAD_DEV',
 'TMP_STP_BDY_AVG',
 'TMP_STP_BDY_MIN',
 'TMP_STP_BDY_MAX',
 'TMP_STP_BDY_DEV',
 'TMP_STP_TAIL_AVG',
 'TMP_STP_TAIL_MIN',
 'TMP_STP_TAIL_MAX',
 'TMP_STP_TAIL_DEV',
 'TMP_STP_ALL_MIN',
 'TMP_STP_ALL_MAX',
 'TMP_STP_ALL_DEV','CR_ACT', 'DIF_TMP_ALL','CR',
 'DEF_CR', 'DIF_STRT_HED_AVG',
 'DIF_STRT_TAIL_AVG',
 'DIF_STRT_TOT','PODKAT',
 'PODKAT2', 'TEMP_START_PHASE2','HEATING_TIME',
          'TMP_STRT_TOT_AVG_M',
 'TMP_STRT_BDY_AVG_M','TMP_STA_TOT_MIN_M', 'TMP_STRT_BDY_AVG','TMP_STRT_ALL_AVG', 'TMP_STRT_TOT_MAX_M',
           'THC_out_degree_sum_act_mean',

 'TMP_STRT_ALL_MIN',
 'TMP_STRT_BDY_MIN','L_ACC',
'hour',
 'UNIFORMITY',
 'BATCH_SIZE',
 'TEMP_OVERHEATING', 'WBF_NO', 'FURT10',
 'FURT40','AVG_MASS_TEMP','CR_HEAD', 'SCC._SccRtData.SccRtData.SpecFlw_TOP_SUM_mean','EDGE_MASKING_POS_DRIVE_SIDE',
 'EDGE_MASKING_POS_OP_SIDE',

 'CR_BDY','WATER_FLOW_SPRAY_TOP',
 'WATER_FLOW_SPRAY_BOTTOM',
 'WATER_FLOW_LAM_INNER_TOP',
 'WATER_FLOW_LAM_OUTER_TOP',
 'WATER_FLOW_LAM_INNER_BOT','TFM',
 'TS',
    
 'WATER_FLOW_LAM_OUTER_BOT', 'ADAPT_CR_PREV',
 'CR_TAIL',
 'CR_SPRAY_HEAD',
 'CR_SPRAY_BDY',
 'CR_SPRAY_TAIL',
 'CR_LAM_HEAD',
 'CR_LAM_BDY',
          'BHC_out_act_degree_sum_mean', 'THC_out_degree_sum_act_mean', 'HEAD_MASKING_TOP_LENGTH',
 'HEAD_MASKING_TOP_FLOW', 'TAIL_MASKING_TOP_LENGTH', 'HEAD_MASKING_BOTTOM_LENGTH',
 'HEAD_MASKING_BOTTOM_FLOW',u'TMP_STRT_BDY_MAX','TMP_STRT_TAIL_MAX',u'TMP_STRT_TAIL_MIN',u'TMP_STRT_BDY_DEV',u'TMP_STRT_ALL_MAX', 'TMP_STRT_HEAD_AVG',u'TMP_STRT_TAIL_AVG',u'TMP_STRT_HEAD_MIN', u'TMP_STRT_HEAD_DEV', 
 'TAIL_MASKING_TOP_FLOW','TOP_FLW_SCS',
 'TAIL_MASKING_BOTTOM_LENGTH',
 'TAIL_MASKING_BOTTOM_FLOW',  'BANKS_HEIGHT', 'MEAS_TIME',u'DIF_SPRAY_BDY_HEAD', u'DIF_SPRAY_BDY_TAIL',
  u'DIF_LAM_BDY_TAIL',
 'PREV_MEAS_TIME',
 'SIDE_SPRAY',
 'AIR_BLOWER',u'PLT_COOL_SPD_AVG', u'PLT_COOL_SPD_HEAD_AVG',
 'NOT_IN_40_0C',
 'IS_FIRST',
 'WATER_TEMP_LAMINAR_COOLING', 'NOT_ANALYSIS',u'DIF_BDY_TAIL', u'DIF_BDY_HEAD',   u'SPEED',  u'DIF_LAM_BDY_HEAD',

 'CR_LAM_TAIL']

df.drop([x for x in df.columns if 'mean_mean' in x  or 'mean_act_mean' in x or 'mean_task_mean' in x or 'task' in x or x in todrop], axis=1, inplace=1)

In [5]:
for col in [
 'ADAPTATION_KEY',
'HEADERS',
'WBF_COL_NO',
 'BATCH_POSITION','ADAPTION_KEY_CHANGED',
'daypart']:
    df[col] = df[col].apply(lambda x: str(x))

In [6]:
categorical_columns = [c for c in df.columns if df[c].dtype.name == 'object']
numerical_columns   = [c for c in df.columns if df[c].dtype.name != 'object' and c not in ['NOT_IN_25_0C','TMP_STP_ALL_AVG','PIECE_ID','prev_fail','outcome']]
print 'Categorical columns:\n', categorical_columns, '\n\n',
print 'Numerical columns:\n', numerical_columns

Categorical columns:
['ADAPTATION_KEY', 'HEADERS', 'WBF_COL_NO', 'BATCH_POSITION', 'ADAPTION_KEY_CHANGED', 'daypart'] 

Numerical columns:
['time', 'GFC._Gfc2Hmi.Gfc2Hmi.GfcHmiTmpWatLpc_ActV_mean', 'GFC._Gfc2Hmi.Gfc2Hmi.GfcHmiGenWtsAirTmp_ActV_mean', 'GFC.Ai._aiGfcAirPrs_mean', 'GFC.GlobVar.GfcLocRtData.GfcSccWatPrsScc_mean', 'BHC._BhcRtData.BhcRtData.WatAbsFlwSum_mean', 'PRC.GlobVar.PrcLocRtData.PrcPos_DsOsdiff_sum_mean', 'PRC.GlobVar.PrcLocRtData.PrcPosDiff_sum_mean', 'SCC._SccRtData.SccRtData.SpecFlw_Bot_SUM_mean', 'BHC_cen_act_degree_sum_mean', 'BHC_cen_flow_sum_act_mean', 'BHC_out_act_sum_mean', 'THC_cen_act_degree_sum_mean', 'THC_cen_flow_sum_act_mean', 'THC_out_flow_sum_act_mean', 'RES_TIM_STRT_COMPL', 'TMP_STRT_HEAD_MAX', 'TMP_STRT_TAIL_DEV', 'TMP_STRT_ALL_DEV', 'PLT_COOL_SPD_TAIL_AVG', 'PLT_ACCELERATION', 'ADAPT_CR', 'ADAPT_TS_MEAN', 'TOP_FLW_LPC', 'DURATION_IN_HOLDING_ZONE', 'TEMP_START_PHASE1', 'TEMP_END_PHASE2', 'DIF_TEMP_START', 'DIF_TFM', 'DIF_STRT_TOT_AVG', 'DIF_BDY_HEAD

In [7]:
data_describe = df.describe(include=[object])

binary_columns    = [c for c in categorical_columns if data_describe[c]['unique'] == 2]
nonbinary_columns = [c for c in categorical_columns if data_describe[c]['unique'] > 2]

print binary_columns, nonbinary_columns

['ADAPTATION_KEY', 'WBF_COL_NO', 'ADAPTION_KEY_CHANGED'] ['HEADERS', 'BATCH_POSITION', 'daypart']


In [8]:
for c in binary_columns:
    top = data_describe[c]['top']
    top_items = df[c] == top
    df.loc[top_items, c] = 0
    df.loc[np.logical_not(top_items), c] = 1
    
data_nonbinary = pd.get_dummies(df[nonbinary_columns])

In [9]:
data_numerical = df[numerical_columns]
# data_numerical = (data_numerical - data_numerical.mean()) / data_numerical.std()
# data_numerical.describe()

,time,GFC._Gfc2Hmi.Gfc2Hmi.GfcHmiTmpWatLpc_ActV_mean,GFC._Gfc2Hmi.Gfc2Hmi.GfcHmiGenWtsAirTmp_ActV_mean,GFC.Ai._aiGfcAirPrs_mean,GFC.GlobVar.GfcLocRtData.GfcSccWatPrsScc_mean,BHC._BhcRtData.BhcRtData.WatAbsFlwSum_mean,PRC.GlobVar.PrcLocRtData.PrcPos_DsOsdiff_sum_mean,PRC.GlobVar.PrcLocRtData.PrcPosDiff_sum_mean,SCC._SccRtData.SccRtData.SpecFlw_Bot_SUM_mean,BHC_cen_act_degree_sum_mean,...,diff_mins,CR_PREV,TMP_STP_ALL_PREV,ADAPT_TS_MEAN_PREV,Specflow_bot_sum_diff,Specflow_top_sum_diff,BHC_cen_flow_diff,BHC_out_flow_diff,THC_cen_flow_diff,THC_out_flow_mean
count,1.892000e+03,1.892000e+03,1.892000e+03,1.892000e+03,1.892000e+03,1.892000e+03,1.892000e+03,1.892000e+03,1.892000e+03,1.892000e+03,...,1.892000e+03,1.892000e+03,1.892000e+03,1.892000e+03,1.892000e+03,1.892000e+03,1.892000e+03,1.892000e+03,1.892000e+03,1.892000e+03
mean,-2.108485e-15,7.285046e-15,5.852378e-15,-1.901669e-14,6.160311e-13,-4.817617e-15,-1.955800e-15,-8.615378e-16,1.816024e-15,1.863672e-16,...,-3.854093e-16,4.356980e-18,9.539879e-15,5.830959e-14,-3.482650e-17,2.583088e-16,-5.654157e-15,6.208330e-16,3.083627e-16,1.455613e-15
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-3.600887e+00,-6.340212e+00,-3.311131e+00,-5.311734e+00,-6.147143e+00,-3.602978e+00,-1.725893e+00,-1.112299e+00,-2.648417e+00,-3.670465e+00,...,-6.111769e-01,-8.215322e+00,-6.160905e+00,-4.482464e+00,-4.202603e+01,-4.041947e+01,-2.910092e+00,-1.576500e+00,-3.557435e+00,-7.340142e+00
25%,-1.010141e+00,-2.428437e-02,-5.873574e-01,-6.593834e-01,-2.581838e-01,-9.134478e-01,-9.354165e-01,-8.936078e-01,-1.152501e+00,-9.257952e-01,...,-5.706710e-01,-6.068148e-01,-6.900845e-01,-7.328266e-01,-1.027906e-01,-1.776984e-01,-1.002467e+00,-9.606710e-01,-5.666671e-01,-6.951808e-01
50%,6.768561e-01,2.368459e-01,1.903989e-01,-4.580523e-02,3.836448e-01,-9.027532e-02,-5.587874e-01,-7.460491e-01,-3.893196e-01,-1.260095e-01,...,-5.375298e-01,7.336355e-02,-4.383430e-02,-9.729481e-02,-8.140318e-02,-1.527731e-01,7.069172e-01,-6.663728e-01,1.672518e-01,-1.764215e-02
75%,9.781057e-01,4.478927e-01,7.976892e-01,5.819728e-01,4.466140e-01,9.284684e-01,9.113517e-01,1.160154e+00,9.784481e-01,9.366030e-01,...,7.770713e-01,6.744514e-01,6.020305e-01,7.924497e-01,2.010016e-01,4.876873e-01,9.724012e-01,9.982347e-01,5.409297e-01,7.030351e-01
max,2.484353e+00,9.740778e-01,1.713360e+00,2.908737e+00,6.123697e+00,3.174644e+00,2.670376e+00,1.643456e+00,2.296643e+00,3.261962e+00,...,3.429841e+00,3.094621e+00,6.634837e+00,3.112141e+00,3.380531e+00,3.595613e+00,2.344619e+00,3.120335e+00,6.161671e+00,2.606365e+00


In [10]:
cols =[u'time', u'GFC._Gfc2Hmi.Gfc2Hmi.GfcHmiTmpWatLpc_ActV_mean',
       u'GFC._Gfc2Hmi.Gfc2Hmi.GfcHmiGenWtsAirTmp_ActV_mean',
       u'GFC.Ai._aiGfcAirPrs_mean',
       u'GFC.GlobVar.GfcLocRtData.GfcSccWatPrsScc_mean',
       u'BHC._BhcRtData.BhcRtData.WatAbsFlwSum_mean',
       u'PRC.GlobVar.PrcLocRtData.PrcPos_DsOsdiff_sum_mean',
       u'PRC.GlobVar.PrcLocRtData.PrcPosDiff_sum_mean',
       u'SCC._SccRtData.SccRtData.SpecFlw_Bot_SUM_mean',
       u'BHC_cen_act_degree_sum_mean', u'BHC_cen_flow_sum_act_mean',
       u'BHC_out_act_sum_mean', u'THC_cen_act_degree_sum_mean',
       u'THC_cen_flow_sum_act_mean', u'THC_out_flow_sum_act_mean',
       u'RES_TIM_STRT_COMPL', u'TMP_STRT_HEAD_MAX', u'TMP_STRT_TAIL_DEV',
       u'TMP_STRT_ALL_DEV', u'PLT_COOL_SPD_TAIL_AVG', u'PLT_ACCELERATION',
       u'ADAPT_CR', u'ADAPT_TS_MEAN', u'TOP_FLW_LPC',
       u'DURATION_IN_HOLDING_ZONE', u'TEMP_START_PHASE1', u'TEMP_END_PHASE2',
       u'DIF_TEMP_START', u'DIF_TFM', u'DIF_STRT_TOT_AVG', u'DIF_BDY_HEAD_AVG',
       u'DIF_BDY_TAIL_AVG', u'diff_mins', u'CR_PREV', u'TMP_STP_ALL_PREV',
       u'ADAPT_TS_MEAN_PREV', u'Specflow_bot_sum_diff',
       u'Specflow_top_sum_diff', u'BHC_cen_flow_diff', u'BHC_out_flow_diff',
       u'THC_cen_flow_diff', u'THC_out_flow_mean', u'BATCH_POSITION_1.0',
       u'BATCH_POSITION_2.0', u'BATCH_POSITION_3.0', u'BATCH_POSITION_4.0',
       u'daypart_0', u'daypart_1', u'daypart_2', u'daypart_3', u'daypart_4',
       u'ADAPTATION_KEY', u'HEADERS_23', u'WBF_COL_NO', u'ADAPTION_KEY_CHANGED']
X =  pd.concat((data_numerical,data_nonbinary,df[binary_columns]), axis=1)
X= pd.DataFrame(X[cols], dtype=float)
y = df.TMP_STP_ALL_AVG
y_class = df['NOT_IN_25_0C']
print( X.shape)
print (X.columns)

(1892, 55)
Index([u'time', u'GFC._Gfc2Hmi.Gfc2Hmi.GfcHmiTmpWatLpc_ActV_mean',
       u'GFC._Gfc2Hmi.Gfc2Hmi.GfcHmiGenWtsAirTmp_ActV_mean',
       u'GFC.Ai._aiGfcAirPrs_mean',
       u'GFC.GlobVar.GfcLocRtData.GfcSccWatPrsScc_mean',
       u'BHC._BhcRtData.BhcRtData.WatAbsFlwSum_mean',
       u'PRC.GlobVar.PrcLocRtData.PrcPos_DsOsdiff_sum_mean',
       u'PRC.GlobVar.PrcLocRtData.PrcPosDiff_sum_mean',
       u'SCC._SccRtData.SccRtData.SpecFlw_Bot_SUM_mean',
       u'BHC_cen_act_degree_sum_mean', u'BHC_cen_flow_sum_act_mean',
       u'BHC_out_act_sum_mean', u'THC_cen_act_degree_sum_mean',
       u'THC_cen_flow_sum_act_mean', u'THC_out_flow_sum_act_mean',
       u'RES_TIM_STRT_COMPL', u'TMP_STRT_HEAD_MAX', u'TMP_STRT_TAIL_DEV',
       u'TMP_STRT_ALL_DEV', u'PLT_COOL_SPD_TAIL_AVG', u'PLT_ACCELERATION',
       u'ADAPT_CR', u'ADAPT_TS_MEAN', u'TOP_FLW_LPC',
       u'DURATION_IN_HOLDING_ZONE', u'TEMP_START_PHASE1', u'TEMP_END_PHASE2',
       u'DIF_TEMP_START', u'DIF_TFM', u'DIF_STRT_TOT_AVG', 

In [11]:
X.to_csv('old_train.csv', sep = ';', index=0)
y.to_csv('old_ytrain.csv', sep = ';', index=0,header=1)
y_class.to_csv('old_y_class_train.csv', sep = ';', index=0, header=1)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 11)

N_train, _ = X_train.shape 
N_test,  _ = X_test.shape 
print N_train, N_test

1324 568


In [205]:
X_train, X_test, y_class_train, y_class_test = train_test_split(X, y_class, test_size = 0.3, random_state = 111)

N_train, _ = X_train.shape 
N_test,  _ = X_test.shape 
print N_train, N_test

716 307
